### **Mono Audio Feature Extraction**

In [1]:
from __future__ import print_function
import librosa
import librosa.display
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
file_path = 'Mono_Audio/Fall_Mono'
output_path = 'Mono_Audio_Feature/Fall_Feature'

fall_list = os.listdir(file_path)
count = 1

for file_name in fall_list:
    # 파일 확장자가 .wav인 경우에만 처리
    if file_name.endswith('.wav'):
        full_file_name = os.path.join(file_path, file_name)
        output_file_name = os.path.join(output_path, f'F{count}.csv')
        
        y, sr = librosa.load(full_file_name)
        duration = librosa.get_duration(y=y)
        onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=2205, aggregate=np.median)
        
        pd.set_option('display.max_columns', None)
        df = pd.DataFrame(onset_env)
        df.to_csv(output_file_name)
        
        count = count + 1

In [3]:
file_path = 'Mono_Audio/Non_Fall_Mono'
output_path = 'Mono_Audio_Feature/Non_Fall_Feature'

fall_list = os.listdir(file_path)
count = 1

for file_name in fall_list:
    # 파일 확장자가 .wav인 경우에만 처리
    if file_name.endswith('.wav'):
        full_file_name = os.path.join(file_path, file_name)
        output_file_name = os.path.join(output_path, f'NF{count}.csv')
        
        y, sr = librosa.load(full_file_name)
        duration = librosa.get_duration(y=y)
        onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=2205, aggregate=np.median)
        
        pd.set_option('display.max_columns', None)
        df = pd.DataFrame(onset_env)
        df.to_csv(output_file_name)
        
        count = count + 1

#### Combining Data

In [4]:
# Function to read the second column from CSV files in a directory
def read_second_column_csv(directory):
    dataframes = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            df = pd.read_csv(os.path.join(directory, filename), header=None, usecols=[1])
            flattened_df = df.T
            dataframes.append(flattened_df)
    return pd.concat(dataframes, ignore_index=True)

# Function to apply various augmentations to audio data
def augment_audio_data(y, sr, augmentations=['noise', 'shift', 'stretch', 'pitch']):
    augmented_data = []
    for aug in augmentations:
        if aug == 'noise':
            noise_amp = 0.005 * np.random.uniform() * np.amax(y)
            y_aug = y + noise_amp * np.random.normal(size=y.shape[0])
        elif aug == 'shift':
            shift_range = int(np.random.uniform(low=-5, high=5) * sr)
            y_aug = np.roll(y, shift_range)
        elif aug == 'stretch':
            rate = np.random.uniform(low=0.8, high=1.2)
            y_aug = librosa.effects.time_stretch(y, rate=rate)
        elif aug == 'pitch':
            n_steps = np.random.uniform(low=-1, high=1)
            y_aug = librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)
        augmented_data.append(y_aug)
    return augmented_data


# Function to add augmentations to a dataset
def add_augmentations(directory, sr=16000, augmentations=['noise', 'shift', 'stretch', 'pitch'], repeats=5):
    augmented_dataframes = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            df = pd.read_csv(os.path.join(directory, filename), header=None, usecols=[1])
            y = df.iloc[:, 0].values
            for _ in range(repeats):  # Repeat the augmentations
                augmented_clips = augment_audio_data(y, sr, augmentations)
                for aug_clip in augmented_clips:
                    aug_features = librosa.feature.mfcc(y=aug_clip, sr=sr, n_mfcc=13)
                    flattened_features = np.mean(aug_features, axis=1).reshape(1, -1)
                    augmented_dataframes.append(pd.DataFrame(flattened_features))
    return pd.concat(augmented_dataframes, ignore_index=True)


In [5]:
from sklearn.utils import resample
from imblearn.over_sampling import KMeansSMOTE

In [6]:
fall_dir = 'Mono_Audio_Feature/Fall_Feature'
non_fall_dir = 'Mono_Audio_Feature/Non_Fall_Feature/'

In [7]:
# Read and combine CSV files
falls_data = read_second_column_csv(fall_dir)
not_falls_data = read_second_column_csv(non_fall_dir)

# Apply augmentations
falls_augmented_data = add_augmentations(fall_dir)
not_falls_augmented_data = add_augmentations(non_fall_dir)

# Combine original and augmented data
combined_data = pd.concat([falls_data, not_falls_data, falls_augmented_data, not_falls_augmented_data])
combined_data['label'] = [1] * len(falls_data) + [0] * len(not_falls_data) + \
                         [1] * len(falls_augmented_data) + [0] * len(not_falls_augmented_data)

# Apply K-Means SMOTE for balancing
kmeans_smote = KMeansSMOTE(random_state=123, k_neighbors=10, cluster_balance_threshold=0.1)
X = combined_data.drop('label', axis=1)
y = combined_data['label']
X = X.fillna(0)
X_resampled, y_resampled = kmeans_smote.fit_resample(X, y)

# Combine resampled features and labels into a DataFrame
resampled_data = pd.DataFrame(X_resampled, columns=X.columns)
resampled_data['label'] = y_resampled

# Shuffle the dataset
resampled_data = resampled_data.sample(frac=1, random_state=123).reset_index(drop=True)

# Separate the dataset by label
falls_data_label = resampled_data[resampled_data['label'] == 1]
not_falls_data_label = resampled_data[resampled_data['label'] == 0]

# # Sample 350 entries from each subset
# sampled_falls_data = falls_data_label.sample(n=1200, random_state=123).reset_index(drop=True)
# sampled_not_falls_data = not_falls_data_label.sample(n=1200, random_state=123).reset_index(drop=True)

# Combine the sampled data
combined_sampled_data = pd.concat([falls_data_label, not_falls_data_label]).reset_index(drop=True)

# Shuffle the combined sampled dataset
final_sampled_data = combined_sampled_data.sample(frac=1, random_state=123).reset_index(drop=True)

# Save to CSV
final_sampled_data.to_csv('final_dataset.csv', index=False)

c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=24
  warnings.warn(
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=21
  warnings.warn(
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=28
  warnings.warn(
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=20
  warnings.warn(
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=22
  warnings.warn(
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-package

In [8]:
final_sampled_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,label
0,204.985261,13.479057,5.372032,-2.497969,-9.281000,7.267600,-2.760869,15.405140,17.060632,18.816768,1.234395,14.278804,6.145905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,239.762117,14.515251,32.435932,25.738410,0.312247,-6.601435,14.325894,-10.456043,-8.452368,9.269280,10.007866,20.461684,-2.595440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,254.994708,8.007294,-3.404127,2.070986,-1.262905,0.688634,-0.218533,-0.048011,0.147901,-0.170689,0.155489,-0.122525,0.095177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,211.176332,-0.000128,-0.000040,0.000315,0.000061,0.000192,-0.000250,-0.000212,-0.000321,0.000258,0.000444,0.000362,-0.000525,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,231.482264,22.081993,2.699169,-11.331162,15.852762,-6.295462,4.742458,-3.061402,-1.310794,3.950697,-3.213796,0.314025,2.221359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2436,210.620167,41.492305,-19.536531,5.810097,1.198372,-3.599022,6.709842,-2.508690,4.740055,-3.402757,0.742918,-0.105722,1.109096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.